# What do I need to do for the 565 report?
1. BUild a model - how to build a model?
2. Know how to implement HMM - how?
3. Need to know the structure of data I'm dealing with (decide on pandas or numpy etc) - how?
4. Know how data is stored - how?
5. Extract read counts and store in some form - how?
6. Know how to extract and from where I can extract - how?
7. Think and look up where read counts can be extracted from - how?
8. Look at bam/SAM website & my bam files I have - how?
9. Update my bam files - how?
10. Run my REVISED code (by Clem, just * changed) - how?
11. Go to Beluga and type the command - how?
12. Log into MobaXTerm - how?
13. Bring my diary and type the pwd in - how?
14. Note down the password on my laptop, or I can't access without the diary

In [89]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.estimator import DNNClassifier
#from tensorflow.estimator.inputs import numpy_input_fn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import LinearSVC
import sklearn
import seaborn as sns

In [30]:
fin = "460_ALU_TypeREF_MELT2_GIAB_test-version.csv"

In [31]:
df = pd.read_csv(fin)
df = df.astype(str)
df

,CHR,START,STOP,TR1,TR1_REF,TR1_ALT,TR2,TR2_REF,TR2_ALT,MELT2,MELT2_GL,GIAB
0,1,104719865,104720170,0/1,31,7,0/1,35,12,0/1,"-75.4,-24.08,-317.81",0/1
1,1,145026747,145027058,0/1,116,39,0/1,173,50,0/1,"-174.03,-97.53,-1391.73",0/1
2,1,173944755,173945064,0/1,19,5,0/1,26,17,0/1,"-58,-19.87,-259.01",0/1
3,1,184937054,184937364,0/1,41,15,0/1,52,12,0/0,"-7.83,-31.91,-435.23",0/1
4,1,217917523,217917830,0/1,25,14,0/1,34,12,0/1,"-52.21,-25.89,-372.01",0/1
...,...,...,...,...,...,...,...,...,...,...,...,...
455,X,112877030,112877339,0/1,1,9,1/1,0,7,0/1,"-58,-16.26,-204.01",1/1
456,X,119643326,119643628,1/1,0,16,1/1,0,9,0/1,"-110.2,-25.29,-276.02",1/1
457,X,135157998,135158310,1/1,0,14,1/1,0,14,0/1,"-81.2,-15.65,-144.02",1/1
458,X,138384911,138385220,1/1,0,6,1/1,0,12,0/1,"-34.8,-12.64,-180.01",1/1


In [32]:
#df["merged_name"] = "chr" + df["CHR"] + ":" + df["START"] + "-" + df["STOP"]
df["merged_name"] = "chr" + df["CHR"] + ":" + df["START"].str[:6]
df["merged_name2"] = df["merged_name"].str[:10]
df["merged_name3"] = df["merged_name"].str[:9]
df

,CHR,START,STOP,TR1,TR1_REF,TR1_ALT,TR2,TR2_REF,TR2_ALT,MELT2,MELT2_GL,GIAB,merged_name,merged_name2,merged_name3
0,1,104719865,104720170,0/1,31,7,0/1,35,12,0/1,"-75.4,-24.08,-317.81",0/1,chr1:104719,chr1:10471,chr1:1047
1,1,145026747,145027058,0/1,116,39,0/1,173,50,0/1,"-174.03,-97.53,-1391.73",0/1,chr1:145026,chr1:14502,chr1:1450
2,1,173944755,173945064,0/1,19,5,0/1,26,17,0/1,"-58,-19.87,-259.01",0/1,chr1:173944,chr1:17394,chr1:1739
3,1,184937054,184937364,0/1,41,15,0/1,52,12,0/0,"-7.83,-31.91,-435.23",0/1,chr1:184937,chr1:18493,chr1:1849
4,1,217917523,217917830,0/1,25,14,0/1,34,12,0/1,"-52.21,-25.89,-372.01",0/1,chr1:217917,chr1:21791,chr1:2179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,X,112877030,112877339,0/1,1,9,1/1,0,7,0/1,"-58,-16.26,-204.01",1/1,chrX:112877,chrX:11287,chrX:1128
456,X,119643326,119643628,1/1,0,16,1/1,0,9,0/1,"-110.2,-25.29,-276.02",1/1,chrX:119643,chrX:11964,chrX:1196
457,X,135157998,135158310,1/1,0,14,1/1,0,14,0/1,"-81.2,-15.65,-144.02",1/1,chrX:135157,chrX:13515,chrX:1351
458,X,138384911,138385220,1/1,0,6,1/1,0,12,0/1,"-34.8,-12.64,-180.01",1/1,chrX:138384,chrX:13838,chrX:1383


In [33]:
df_01 = df[df["GIAB"] == "0/1"]
df_01["merged_name3"].to_csv("GIAB_460_10_loci_3.csv", index=False, header=False)

In [34]:
df_11 = df[df["GIAB"] == "1/1"]
df_11["merged_name3"].to_csv("GIAB_460_11_loci_3.csv", index=False, header=False)

In [35]:
df_01

,CHR,START,STOP,TR1,TR1_REF,TR1_ALT,TR2,TR2_REF,TR2_ALT,MELT2,MELT2_GL,GIAB,merged_name,merged_name2,merged_name3
0,1,104719865,104720170,0/1,31,7,0/1,35,12,0/1,"-75.4,-24.08,-317.81",0/1,chr1:104719,chr1:10471,chr1:1047
1,1,145026747,145027058,0/1,116,39,0/1,173,50,0/1,"-174.03,-97.53,-1391.73",0/1,chr1:145026,chr1:14502,chr1:1450
2,1,173944755,173945064,0/1,19,5,0/1,26,17,0/1,"-58,-19.87,-259.01",0/1,chr1:173944,chr1:17394,chr1:1739
3,1,184937054,184937364,0/1,41,15,0/1,52,12,0/0,"-7.83,-31.91,-435.23",0/1,chr1:184937,chr1:18493,chr1:1849
4,1,217917523,217917830,0/1,25,14,0/1,34,12,0/1,"-52.21,-25.89,-372.01",0/1,chr1:217917,chr1:21791,chr1:2179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,21,30601827,30602144,0/1,34,14,0/1,39,12,0/1,"-81.21,-32.51,-435.02",0/1,chr21:306018,chr21:3060,chr21:306
223,21,38961715,38961986,0/1,27,7,0/1,23,3,0/1,"-24.4,-17.46,-264.04",0/1,chr21:389617,chr21:3896,chr21:389
224,21,41586880,41587182,0/1,27,10,0/1,44,9,0/1,"-34.81,-18.66,-249.01",0/1,chr21:415868,chr21:4158,chr21:415
225,21,41957251,41957376,0/1,14,11,0/1,21,12,0/1,"-46.41,-25.29,-346.01",0/1,chr21:419572,chr21:4195,chr21:419


# Part 2

In [69]:
f00 = "GIAB_00_f2.csv"
f10 = "GIAB_10_f2.csv"
f11 = "GIAB_11_f2.csv"

In [70]:
df00 = pd.read_csv(f00, header=None)
df10 = pd.read_csv(f10, header=None)
df11 = pd.read_csv(f11, header=None)
df00 = df00.astype(str)
df10 = df10.astype(str)
df11 = df11.astype(str)
df00

,0
0,chr10:2402963-2403297/ 0/0 210 12 222
1,chr10:51539308-51539401/ 0/0 52 18 70
2,chr10:75969790-75970127/ 0/0 248 22 270
3,chr10:79764973-79765218/ 0/0 188 20 208
4,chr10:8516524-8516843/ 0/0 218 22 240
...,...
95,chr9:77503386-77503546/ 0/0 96 46 142
96,chrX:113850728-113851049/ 0/0 84 12 96
97,chrX:43238081-43238371/ 0/0 74 12 86
98,chrX:54290473-54290777/ 0/0 112 4 116


In [71]:
# split column into multiple columns by delimiter 
df00 = df00[0].str.split(' ', expand=True)
df10 = df10[0].str.split(' ', expand=True)
df11 = df11[0].str.split(' ', expand=True)

df00

,0,1,2,3,4
0,chr10:2402963-2403297/,0/0,210,12,222
1,chr10:51539308-51539401/,0/0,52,18,70
2,chr10:75969790-75970127/,0/0,248,22,270
3,chr10:79764973-79765218/,0/0,188,20,208
4,chr10:8516524-8516843/,0/0,218,22,240
...,...,...,...,...,...
95,chr9:77503386-77503546/,0/0,96,46,142
96,chrX:113850728-113851049/,0/0,84,12,96
97,chrX:43238081-43238371/,0/0,74,12,86
98,chrX:54290473-54290777/,0/0,112,4,116


In [72]:
df00.columns = ["dir_name", "GIAB_genotype", "f2_count_REF", "f2_count_ALT", "f2_count"]
df10.columns = ["dir_name", "GIAB_genotype", "f2_count_REF", "f2_count_ALT", "f2_count"]
df11.columns = ["dir_name", "GIAB_genotype", "f2_count_REF", "f2_count_ALT", "f2_count"]
df00

,dir_name,GIAB_genotype,f2_count_REF,f2_count_ALT,f2_count
0,chr10:2402963-2403297/,0/0,210,12,222
1,chr10:51539308-51539401/,0/0,52,18,70
2,chr10:75969790-75970127/,0/0,248,22,270
3,chr10:79764973-79765218/,0/0,188,20,208
4,chr10:8516524-8516843/,0/0,218,22,240
...,...,...,...,...,...
95,chr9:77503386-77503546/,0/0,96,46,142
96,chrX:113850728-113851049/,0/0,84,12,96
97,chrX:43238081-43238371/,0/0,74,12,86
98,chrX:54290473-54290777/,0/0,112,4,116


In [73]:
dict_type_convert = {"f2_count_REF": int, "f2_count_ALT": int, "f2_count": int}
df00 = df00.astype(dict_type_convert)
df10 = df10.astype(dict_type_convert)
df11 = df11.astype(dict_type_convert)
# print(df00.dtypes)

In [74]:
frames = [df00, df10, df11]
df_560 = pd.concat(frames)
# df_560 = pd.concat(frames, keys=["dir_name", "GIAB_genotype", "cF260_count"])
# Remove the '/' at the end of directory name
df_560["dir_name"] = df_560["dir_name"].str.split("/").str[0]
df_560

,dir_name,GIAB_genotype,f2_count_REF,f2_count_ALT,f2_count
0,chr10:2402963-2403297,0/0,210,12,222
1,chr10:51539308-51539401,0/0,52,18,70
2,chr10:75969790-75970127,0/0,248,22,270
3,chr10:79764973-79765218,0/0,188,20,208
4,chr10:8516524-8516843,0/0,218,22,240
...,...,...,...,...,...
231,chrX:112877016-112877360,1/1,24,38,62
232,chrX:119643312-119643638,1/1,24,62,86
233,chrX:135157983-135158330,1/1,24,46,70
234,chrX:138384895-138385237,1/1,16,44,60


In [75]:
df_560.groupby(by="GIAB_genotype")["f2_count_REF"].mean()

GIAB_genotype
0/0    171.900000
1/0     90.348214
1/1     45.737288
Name: f2_count_REF, dtype: float64

In [76]:
df_560["GIAB_genotype"] = df_560["GIAB_genotype"].map({"0/0":0,
                             "1/0":1,
                             "1/1":2,
                             np.nan:3},
                             na_action=None)
df_560

,dir_name,GIAB_genotype,f2_count_REF,f2_count_ALT,f2_count
0,chr10:2402963-2403297,0,210,12,222
1,chr10:51539308-51539401,0,52,18,70
2,chr10:75969790-75970127,0,248,22,270
3,chr10:79764973-79765218,0,188,20,208
4,chr10:8516524-8516843,0,218,22,240
...,...,...,...,...,...
231,chrX:112877016-112877360,2,24,38,62
232,chrX:119643312-119643638,2,24,62,86
233,chrX:135157983-135158330,2,24,46,70
234,chrX:138384895-138385237,2,16,44,60


In [77]:
from sklearn import datasets

In [78]:
iris = datasets.load_iris()
X = iris.data[:, :]
y = iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [82]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [101]:
y = df_560["GIAB_genotype"].to_numpy()
#y

In [86]:
X = df_560.iloc[:, 2:].to_numpy()

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
                        X, y, test_size=0.2, random_state=42)
## Check out what "random_state=42" represents

In [100]:
clf = LinearSVC(penalty="l2", loss="squared_hinge", 
                dual=True, tol=0.0001, C=100, multi_class="ovr",
                fit_intercept=True, intercept_scaling=1, class_weight=None,
                verbose=0, random_state=0, max_iter=100000)
clf.fit(X_train, y_train)

print('Accuracy of linear SVC on training set: {:.3f}'.format(clf.score(X_train, y_train)))

print('Accuracy of linear SVC on test set: {:.3f}'.format(clf.score(X_test, y_test)))

Accuracy of linear SVC on training set: 0.766
Accuracy of linear SVC on test set: 0.705


C:\Users\Yumika\anaconda3-2\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [102]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [115]:
encoder = LabelBinarizer()
seed = 42

df_GIAB_2_X = pd.DataFrame(data=X, columns=["f2_count_REF", "f2_count_ALT", "f2_count"],
                       dtype=np.float32)
df_GIAB_2_X

,f2_count_REF,f2_count_ALT,f2_count
0,210.0,12.0,222.0
1,52.0,18.0,70.0
2,248.0,22.0,270.0
3,188.0,20.0,208.0
4,218.0,22.0,240.0
...,...,...,...
555,24.0,38.0,62.0
556,24.0,62.0,86.0
557,24.0,46.0,70.0
558,16.0,44.0,60.0


In [116]:
target = encoder.fit_transform(y)
df_GIAB_2_y = pd.DataFrame(data=target, columns=["0/0", "1/0", "1/1"])
df_GIAB_2_y

,0/0,1/0,1/1
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
555,0,0,1
556,0,0,1
557,0,0,1
558,0,0,1


In [118]:
X_train,X_test,y_train,y_test = train_test_split(df_GIAB_2_X,
                                                 df_GIAB_2_y,
                                                 test_size=0.20,
                                                 random_state=seed)
 
scaler = MinMaxScaler(feature_range=(0,1))
 
X_train = pd.DataFrame(scaler.fit_transform(X_train),
                               columns=X_train.columns,
                               index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test),
                           columns=X_test.columns,
                           index=X_test.index)
 
def model():
    """build the Keras model callback"""
    model = Sequential()
    model.add(Dense(6, input_dim=3, activation='tanh', name='layer_1'))
    model.add(Dense(10, activation='tanh', name='layer_2'))
    model.add(Dense(3, activation='softmax', name='output_layer'))
     
    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])
    return model
 
estimator = KerasClassifier(
        build_fn=model,
        epochs=200, batch_size=20,
        verbose=2)
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Model Performance: mean: %.2f%% std: (%.2f%%)" % (results.mean()*100, results.std()*100))

<ipython-input-118-7e0e186bb04d>:27: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator = KerasClassifier(


Epoch 1/200
18/18 - 1s - loss: 1.0772 - accuracy: 0.4078 - 1s/epoch - 60ms/step
Epoch 2/200
18/18 - 0s - loss: 1.0461 - accuracy: 0.3883 - 59ms/epoch - 3ms/step
Epoch 3/200
18/18 - 0s - loss: 1.0199 - accuracy: 0.3492 - 57ms/epoch - 3ms/step
Epoch 4/200
18/18 - 0s - loss: 0.9998 - accuracy: 0.3939 - 58ms/epoch - 3ms/step
Epoch 5/200
18/18 - 0s - loss: 0.9821 - accuracy: 0.4385 - 58ms/epoch - 3ms/step
Epoch 6/200
18/18 - 0s - loss: 0.9655 - accuracy: 0.4385 - 58ms/epoch - 3ms/step
Epoch 7/200
18/18 - 0s - loss: 0.9484 - accuracy: 0.4497 - 64ms/epoch - 4ms/step
Epoch 8/200
18/18 - 0s - loss: 0.9301 - accuracy: 0.4385 - 36ms/epoch - 2ms/step
Epoch 9/200
18/18 - 0s - loss: 0.9107 - accuracy: 0.4469 - 45ms/epoch - 2ms/step
Epoch 10/200
18/18 - 0s - loss: 0.8881 - accuracy: 0.4832 - 34ms/epoch - 2ms/step
Epoch 11/200
18/18 - 0s - loss: 0.8632 - accuracy: 0.5279 - 33ms/epoch - 2ms/step
Epoch 12/200
18/18 - 0s - loss: 0.8362 - accuracy: 0.5307 - 32ms/epoch - 2ms/step
Epoch 13/200
18/18 - 0s - 

18/18 - 0s - loss: 0.3450 - accuracy: 0.8994 - 32ms/epoch - 2ms/step
Epoch 102/200
18/18 - 0s - loss: 0.3441 - accuracy: 0.8994 - 31ms/epoch - 2ms/step
Epoch 103/200
18/18 - 0s - loss: 0.3438 - accuracy: 0.8994 - 30ms/epoch - 2ms/step
Epoch 104/200
18/18 - 0s - loss: 0.3429 - accuracy: 0.8994 - 33ms/epoch - 2ms/step
Epoch 105/200
18/18 - 0s - loss: 0.3427 - accuracy: 0.8994 - 37ms/epoch - 2ms/step
Epoch 106/200
18/18 - 0s - loss: 0.3420 - accuracy: 0.8966 - 33ms/epoch - 2ms/step
Epoch 107/200
18/18 - 0s - loss: 0.3415 - accuracy: 0.8966 - 35ms/epoch - 2ms/step
Epoch 108/200
18/18 - 0s - loss: 0.3409 - accuracy: 0.9022 - 33ms/epoch - 2ms/step
Epoch 109/200
18/18 - 0s - loss: 0.3400 - accuracy: 0.8994 - 31ms/epoch - 2ms/step
Epoch 110/200
18/18 - 0s - loss: 0.3394 - accuracy: 0.8994 - 34ms/epoch - 2ms/step
Epoch 111/200
18/18 - 0s - loss: 0.3388 - accuracy: 0.8994 - 38ms/epoch - 2ms/step
Epoch 112/200
18/18 - 0s - loss: 0.3397 - accuracy: 0.9022 - 33ms/epoch - 2ms/step
Epoch 113/200
18/1

Epoch 200/200
18/18 - 0s - loss: 0.2789 - accuracy: 0.9022 - 34ms/epoch - 2ms/step
5/5 - 0s - loss: 0.3881 - accuracy: 0.8667 - 351ms/epoch - 70ms/step
Epoch 1/200
18/18 - 1s - loss: 1.0712 - accuracy: 0.6704 - 843ms/epoch - 47ms/step
Epoch 2/200
18/18 - 0s - loss: 1.0447 - accuracy: 0.7486 - 34ms/epoch - 2ms/step
Epoch 3/200
18/18 - 0s - loss: 1.0239 - accuracy: 0.7346 - 32ms/epoch - 2ms/step
Epoch 4/200
18/18 - 0s - loss: 1.0052 - accuracy: 0.7346 - 33ms/epoch - 2ms/step
Epoch 5/200
18/18 - 0s - loss: 0.9878 - accuracy: 0.6592 - 31ms/epoch - 2ms/step
Epoch 6/200
18/18 - 0s - loss: 0.9700 - accuracy: 0.6676 - 34ms/epoch - 2ms/step
Epoch 7/200
18/18 - 0s - loss: 0.9496 - accuracy: 0.6564 - 34ms/epoch - 2ms/step
Epoch 8/200
18/18 - 0s - loss: 0.9292 - accuracy: 0.5978 - 33ms/epoch - 2ms/step
Epoch 9/200
18/18 - 0s - loss: 0.9057 - accuracy: 0.6061 - 32ms/epoch - 2ms/step
Epoch 10/200
18/18 - 0s - loss: 0.8813 - accuracy: 0.6257 - 34ms/epoch - 2ms/step
Epoch 11/200
18/18 - 0s - loss: 0.8

Epoch 100/200
18/18 - 0s - loss: 0.3859 - accuracy: 0.8799 - 35ms/epoch - 2ms/step
Epoch 101/200
18/18 - 0s - loss: 0.3858 - accuracy: 0.8799 - 31ms/epoch - 2ms/step
Epoch 102/200
18/18 - 0s - loss: 0.3848 - accuracy: 0.8799 - 31ms/epoch - 2ms/step
Epoch 103/200
18/18 - 0s - loss: 0.3844 - accuracy: 0.8827 - 33ms/epoch - 2ms/step
Epoch 104/200
18/18 - 0s - loss: 0.3841 - accuracy: 0.8799 - 34ms/epoch - 2ms/step
Epoch 105/200
18/18 - 0s - loss: 0.3826 - accuracy: 0.8799 - 32ms/epoch - 2ms/step
Epoch 106/200
18/18 - 0s - loss: 0.3830 - accuracy: 0.8799 - 36ms/epoch - 2ms/step
Epoch 107/200
18/18 - 0s - loss: 0.3852 - accuracy: 0.8799 - 34ms/epoch - 2ms/step
Epoch 108/200
18/18 - 0s - loss: 0.3806 - accuracy: 0.8799 - 35ms/epoch - 2ms/step
Epoch 109/200
18/18 - 0s - loss: 0.3813 - accuracy: 0.8827 - 34ms/epoch - 2ms/step
Epoch 110/200
18/18 - 0s - loss: 0.3800 - accuracy: 0.8827 - 36ms/epoch - 2ms/step
Epoch 111/200
18/18 - 0s - loss: 0.3802 - accuracy: 0.8827 - 35ms/epoch - 2ms/step
Epoc

...

Epoch 198/200
18/18 - 0s - loss: 0.2968 - accuracy: 0.8994 - 30ms/epoch - 2ms/step
Epoch 199/200
18/18 - 0s - loss: 0.2956 - accuracy: 0.8994 - 33ms/epoch - 2ms/step
Epoch 200/200
18/18 - 0s - loss: 0.2944 - accuracy: 0.9022 - 31ms/epoch - 2ms/step
5/5 - 0s - loss: 0.3176 - accuracy: 0.9000 - 298ms/epoch - 60ms/step
Epoch 1/200
18/18 - 1s - loss: 1.0897 - accuracy: 0.2145 - 1s/epoch - 60ms/step
Epoch 2/200
18/18 - 0s - loss: 1.0207 - accuracy: 0.5292 - 36ms/epoch - 2ms/step
Epoch 3/200
18/18 - 0s - loss: 0.9657 - accuracy: 0.5515 - 34ms/epoch - 2ms/step
Epoch 4/200
18/18 - 0s - loss: 0.9209 - accuracy: 0.5543 - 40ms/epoch - 2ms/step
Epoch 5/200
18/18 - 0s - loss: 0.8833 - accuracy: 0.5766 - 39ms/epoch - 2ms/step
Epoch 6/200
18/18 - 0s - loss: 0.8483 - accuracy: 0.5933 - 39ms/epoch - 2ms/step
Epoch 7/200
18/18 - 0s - loss: 0.8156 - accuracy: 0.6184 - 43ms/epoch - 2ms/step
Epoch 8/200
18/18 - 0s - loss: 0.7884 - accuracy: 0.6630 - 45ms/epoch - 2ms/step
Epoch 9/200
18/18 - 0s - loss: 0.76

18/18 - 0s - loss: 0.3516 - accuracy: 0.8942 - 31ms/epoch - 2ms/step
Epoch 98/200
18/18 - 0s - loss: 0.3513 - accuracy: 0.8914 - 28ms/epoch - 2ms/step
Epoch 99/200
18/18 - 0s - loss: 0.3493 - accuracy: 0.8942 - 30ms/epoch - 2ms/step
Epoch 100/200
18/18 - 0s - loss: 0.3494 - accuracy: 0.8942 - 32ms/epoch - 2ms/step
Epoch 101/200
18/18 - 0s - loss: 0.3464 - accuracy: 0.8942 - 28ms/epoch - 2ms/step
Epoch 102/200
18/18 - 0s - loss: 0.3459 - accuracy: 0.8942 - 28ms/epoch - 2ms/step
Epoch 103/200
18/18 - 0s - loss: 0.3445 - accuracy: 0.8942 - 28ms/epoch - 2ms/step
Epoch 104/200
18/18 - 0s - loss: 0.3430 - accuracy: 0.8942 - 31ms/epoch - 2ms/step
Epoch 105/200
18/18 - 0s - loss: 0.3424 - accuracy: 0.8942 - 28ms/epoch - 2ms/step
Epoch 106/200
18/18 - 0s - loss: 0.3409 - accuracy: 0.8942 - 30ms/epoch - 2ms/step
Epoch 107/200
18/18 - 0s - loss: 0.3396 - accuracy: 0.8942 - 30ms/epoch - 2ms/step
Epoch 108/200
18/18 - 0s - loss: 0.3388 - accuracy: 0.8942 - 32ms/epoch - 2ms/step
Epoch 109/200
18/18 

Epoch 196/200
18/18 - 0s - loss: 0.2700 - accuracy: 0.8969 - 26ms/epoch - 1ms/step
Epoch 197/200
18/18 - 0s - loss: 0.2694 - accuracy: 0.8969 - 27ms/epoch - 2ms/step
Epoch 198/200
18/18 - 0s - loss: 0.2686 - accuracy: 0.8969 - 32ms/epoch - 2ms/step
Epoch 199/200
18/18 - 0s - loss: 0.2693 - accuracy: 0.8969 - 31ms/epoch - 2ms/step
Epoch 200/200
18/18 - 0s - loss: 0.2686 - accuracy: 0.8969 - 30ms/epoch - 2ms/step
5/5 - 0s - loss: 0.2713 - accuracy: 0.8989 - 286ms/epoch - 57ms/step
Epoch 1/200
18/18 - 1s - loss: 1.2834 - accuracy: 0.1811 - 915ms/epoch - 51ms/step
Epoch 2/200
18/18 - 0s - loss: 1.1980 - accuracy: 0.1950 - 40ms/epoch - 2ms/step
Epoch 3/200
18/18 - 0s - loss: 1.1446 - accuracy: 0.1616 - 38ms/epoch - 2ms/step
Epoch 4/200
18/18 - 0s - loss: 1.1044 - accuracy: 0.3649 - 49ms/epoch - 3ms/step
Epoch 5/200
18/18 - 0s - loss: 1.0775 - accuracy: 0.4290 - 47ms/epoch - 3ms/step
Epoch 6/200
18/18 - 0s - loss: 1.0545 - accuracy: 0.4290 - 44ms/epoch - 2ms/step
Epoch 7/200
18/18 - 0s - los

Epoch 96/200
18/18 - 0s - loss: 0.3859 - accuracy: 0.8886 - 31ms/epoch - 2ms/step
Epoch 97/200
18/18 - 0s - loss: 0.3859 - accuracy: 0.8858 - 35ms/epoch - 2ms/step
Epoch 98/200
18/18 - 0s - loss: 0.3845 - accuracy: 0.8886 - 33ms/epoch - 2ms/step
Epoch 99/200
18/18 - 0s - loss: 0.3846 - accuracy: 0.8858 - 40ms/epoch - 2ms/step
Epoch 100/200
18/18 - 0s - loss: 0.3837 - accuracy: 0.8886 - 33ms/epoch - 2ms/step
Epoch 101/200
18/18 - 0s - loss: 0.3833 - accuracy: 0.8858 - 32ms/epoch - 2ms/step
Epoch 102/200
18/18 - 0s - loss: 0.3824 - accuracy: 0.8858 - 33ms/epoch - 2ms/step
Epoch 103/200
18/18 - 0s - loss: 0.3824 - accuracy: 0.8886 - 36ms/epoch - 2ms/step
Epoch 104/200
18/18 - 0s - loss: 0.3829 - accuracy: 0.8858 - 32ms/epoch - 2ms/step
Epoch 105/200
18/18 - 0s - loss: 0.3816 - accuracy: 0.8886 - 32ms/epoch - 2ms/step
Epoch 106/200
18/18 - 0s - loss: 0.3808 - accuracy: 0.8886 - 36ms/epoch - 2ms/step
Epoch 107/200
18/18 - 0s - loss: 0.3804 - accuracy: 0.8886 - 35ms/epoch - 2ms/step
Epoch 10

Epoch 195/200
18/18 - 0s - loss: 0.3114 - accuracy: 0.8914 - 33ms/epoch - 2ms/step
Epoch 196/200
18/18 - 0s - loss: 0.3087 - accuracy: 0.8914 - 34ms/epoch - 2ms/step
Epoch 197/200
18/18 - 0s - loss: 0.3079 - accuracy: 0.8914 - 31ms/epoch - 2ms/step
Epoch 198/200
18/18 - 0s - loss: 0.3073 - accuracy: 0.8914 - 29ms/epoch - 2ms/step
Epoch 199/200
18/18 - 0s - loss: 0.3069 - accuracy: 0.8914 - 33ms/epoch - 2ms/step
Epoch 200/200
18/18 - 0s - loss: 0.3058 - accuracy: 0.8914 - 30ms/epoch - 2ms/step
5/5 - 0s - loss: 0.3133 - accuracy: 0.8764 - 236ms/epoch - 47ms/step
Model Performance: mean: 89.06% std: (1.64%)
